# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [27]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [14]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## What test is appropriate for this problem? Does CLT apply?



In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

We can use A/B testing since we have two groups in this dataset. One is for White sounding name and the other one is Black sounding name. We can compare these two and find out which one is more likely to get a call from a company for job.

In [42]:
# number of callbacks for white-sounding names
print("White" , sum(data[data.race=='w'].call))
# number of callbacks for black-sounding names
print("Black" , sum(data[data.race=='b'].call))
white_names =  data[data['race']=='w']
black_names = data[data['race']=='b']
print("Total white sounding applications ", len(white_names))
print("Total black sounding applications ", len(black_names))
print("Total applications " , len(data))

White 235.0
Black 157.0
Total white sounding applications  2435
Total black sounding applications  2435
Total applications  4870


## What are the null and alternate hypotheses?

#### Null Hypothesis 
There is no difference between black sounding name and white sounding name to receive a call for a job.

#### Alternative Hypothesis
There is a difference.

## Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [34]:
w = data[data.race=='w']
b = data[data.race=='b']

In [48]:
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data, size=len(data)))

    return bs_replicates

In [83]:
margin_error = np.std(w.call) / np.sqrt(len(w))
print("Margin error for white sounding names " , margin_error)
margin_error1 = np.std(b.call) / np.sqrt(len(b))
print("Margin error for black sounding names " ,margin_error1)
print("Standard deviation for White sounding name ", np.std(w.call))
print("Standard deviation for Black sounding name " , np.std(b.call))
black_confidence = np.percentile(np.mean(b.call) , [2.5,97.5])
white_confidence = np.percentile(np.mean(w.call) , [2.5,97.5])
print("Confidence level for Blacks " , black_confidence , " & White ", white_confidence)

Margin error for white sounding names  0.00598400169818
Margin error for black sounding names  0.0049771088698
Standard deviation for White sounding name  0.29528486728668213
Standard deviation for Black sounding name  0.24559901654720306
Confidence level for Blacks  [ 0.06447639  0.06447639]  & White  [ 0.09650924  0.09650924]


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

### Write a story describing the statistical significance in the context or the original problem.

As we can clearly see that white sounding name are more likely to get a call according to this study. We also know that there are as many applications for black sounding name as many are for white sounding name. It is very clear to see the difference between getting more calls for white sounding name. 

### Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

We know that all the resume are exact same for black sounding name and white sounding name. We do not have to think about other big factors such as education , experience and skills. We can make a conlusion form this study that white sounding name are more likely to get a call for jobs.